# Storing and Accessing objects via Descartes Labs Storage

The Descartes Labs Catalog API also supports arbitrary formats of data that don't fall neatly into either a raster" or vector data model.

This guide covers the basic methods for generic blob storage. For a more in depth overview of all Catalog classes and their capabilities please visit the [API Reference](https://docs.descarteslabs.com/descarteslabs/catalog/readme.html) and [Catalog Guide](https://docs.descarteslabs.com/guides/catalog.html) sections in our Documentation page.

In [ ]:
import descarteslabs as dl
from descarteslabs.catalog import Blob, properties as p

In [ ]:
import json

## Creating a Blob
The only required attribute for a [`Blob`](https://docs.descarteslabs.com/descarteslabs/catalog/docs/blob.html) is a *unique* ID. As with Products, blobs also contain a namespace, defaulting to the user's organization and user ID if none is specified:

    my-org-id:my-user-id

Similar to [02 Creating and Managing Products.ipynb](./02%20Creating%20and%20Managing%20Products.ipynb), we will first format our default namespace with the current user's org and ID:

In [ ]:
org = dl.auth.Auth().payload["org"]
user_namespace = dl.auth.Auth().namespace

## Storing Arbitrary Data as Blobs

Blobs also contain various attributes to search and filter by, such as a geometry and [`tags`](https://docs.descarteslabs.com/descarteslabs/catalog/docs/blob.html#descarteslabs.catalog.Blob.tags). 

In this first example let's define a JSON dictionary of random data and a geometry to go with it:

In [ ]:
# JSON of random field info
crop_info = {
    "crop": "hops",
    "acreage": 450,
}
# Geometry for the field
field_geom = {
    "type": "Polygon",
    "coordinates": [
        [
            [-120.4023, 46.551],
            [-120.3859, 46.551],
            [-120.3859, 46.5534],
            [-120.4023, 46.5534],
        ]
    ],
}

Note we are using JSON for simplicity, however this could be any object format you choose.

Now that we have the data we want to store, we can create create a new blob and call [`Blob.upload_data()`](https://docs.descarteslabs.com/descarteslabs/catalog/docs/blob.html#descarteslabs.catalog.Blob.upload_data) to upload our dataset. Finally, we call [`Blob.save()`](https://docs.descarteslabs.com/descarteslabs/catalog/docs/blob.html#descarteslabs.catalog.Blob.save):

In [ ]:
blob = Blob(
    name="hop_field_info_demo",
    geometry=field_geom,
    tags=["storage_demo"],
)
blob.upload_data(json.dumps(crop_info))
blob.save()
blob

#### _Note on Namespaces_

You can pass any arbitrary string as a specified namespace. Any namespace provided by the user will automatically be prefixed by the user’s organization and a colon if not already present.

While we didn't pass a namespace that information was automatically populated in the blob ID above, defaulting to:

    {my-org}:{my-user-id}

## Storing Files as Blobs
If your object is located on disk, you can instead call [`Blob.upload()`](https://docs.descarteslabs.com/descarteslabs/catalog/docs/blob.html#descarteslabs.catalog.Blob.upload):

In [ ]:
file_blob = Blob(
    name="yakima_valley",
    namespace=f"{org}:{user_namespace}",
    tags=["storage_demo"],
)
file_blob.upload("data/yakima.geojson")
file_blob.save()
file_blob

Notice that the namespaces on these two blobs are identical

## Blob Attributes and Storage Types
The resulting saved blob has several attributes, a few of which are printed below. Note that the ID is the concatenation of the [`StorageType`](https://docs.descarteslabs.com/descarteslabs/catalog/docs/blob.html#descarteslabs.catalog.Blob.storage_type), the namespace, and the name. 

*Note: (Only) The name may contain internal '/' characters.*

If we inspect the ID field we can break the namespace down further:
* *data/* is the storage type
* *my-org-name:my-user-id/* is the namespace
* *hop_field_demo* is the blob's name

In [ ]:
print("Blob ID:", blob.id)
print("Blob size:", blob.size_bytes)
print("Blob geometry: ", blob.geometry)
print("Blob assigned tags: ", blob.tags)

## Searching Blobs
Catalog search methods can be performed across your storage objects, including geospatial searches as well. 

Below are examples of:
* Point intersection
* Polygon intersection
* Tag filter
* Filtering by Storage Type

In [ ]:
# Geospatial searches by intersection
## Intersect particular coordinate
print("Point Intersection:")
print(
    [
        b.id
        for b in Blob.search().intersects(
            {"type": "Point", "coordinates": [-120.40, 46.552]}
        )
    ]
)
print("Polygon Intersectsion:")
## Intersecting geometry object
print([b.id for b in Blob.search().intersects(field_geom)])
print("Tag Filter:")
# Filter by tags
print([b.id for b in Blob.search().filter(p.tags == "storage_demo")])
print("Storage Type Filter:")
# Filter by Storage Type
print([b.id for b in Blob.search().filter(p.storage_type == "data")])

We can also use a prefix filter to pick out these new blobs:

In [ ]:
for b in Blob.search().filter(p.name.prefix("foo/")):
    print(b.id)

### Retrieving data from Blobs
The blob data may be retrieved, either by downloading directly to a local file or some other file-like object (e.g. an io.IOBase object), or directly into memory. Here's a simple download to a file via [`Blob.download()`](https://docs.descarteslabs.com/descarteslabs/catalog/docs/blob.html?highlight=storagetype#descarteslabs.catalog.Blob.download):

In [ ]:
file_blob.download("data/yakima_valley.geojson")

You can also download in raw bytes via [`Blob.data()`](https://docs.descarteslabs.com/descarteslabs/catalog/docs/blob.html?highlight=storagetype#descarteslabs.catalog.Blob.data):

In [ ]:
print(file_blob.data())

### Sharing Blobs
As with Products, you can add specific organizations or users as readers to your blobs to give others access. Simply update the readers list, then saving the blob:

In [ ]:
# Adding coworker and org as readers for previously stored GeoTiff file
file_blob.readers = ["email:john.daily@gmail.com", "org:pga-tour"]
file_blob.save()
file_blob.readers

Blobs can also transfer ownership, however you cannot use the *email:* variant as an owner. Instead use the new owner's user ID:

    file_blob.owners.append(f"user:{user_namespace}")
    file_blob.save()

For more information visit our [Sharing Resources](https://docs.descarteslabs.com/guides/sharing.html) page.

### Deleting Blobs
Blobs can also be deleted using the Catalog module search methods we used earlier and by calling [`Blob.delete()`](https://docs.descarteslabs.com/descarteslabs/catalog/docs/blob.html?highlight=storagetype#descarteslabs.catalog.Blob.delete):

In [ ]:
# Delete by tags
for b in (
    Blob.search()
    .filter(p.tags == "storage_demo")
    .filter(p.owners == f"user:{user_namespace}")
):
    print(f"Deleting {b.id}")
    b.delete()

Cleaning up our workspace

In [ ]:
import os

os.remove("data/yakima_valley.geojson")